In [ ]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
import numpy as np
from tbdynamics.calibration.utils import calculate_covid_diff_quantiles
from tbdynamics.calibration.plotting import plot_covid_scenarios_comparison
import arviz as az
import xarray as xr

In [ ]:
OUT_PATH = Path.cwd() / 'runs/r2508'
idata = az.from_netcdf(OUT_PATH / 'calib_full_out.nc')
burnt_idata = idata.sel(draw=np.s_[50000:])
idata_extract = az.extract(burnt_idata, num_samples=1000)
inference_data = az.convert_to_inference_data(idata_extract.reset_index('sample'))
az.to_netcdf(inference_data, 'inference_data1.nc')

In [ ]:
# loaded_inference_data = az.from_netcdf('inference_data.nc')
# extracted_dataset = xr.Dataset(idata_extract)
# extracted_idata = az.InferenceData(posterior=extracted_dataset)
loaded_inference_data = az.from_netcdf('inference_data1.nc')

In [ ]:
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':True
}

In [ ]:
quantile_ouputs_diff = calculate_covid_diff_quantiles(params, loaded_inference_data, ['cumulative_diseased', 'cumulative_deaths'],[2021.0,2022.0, 2025.0, 2035.0],1)


In [ ]:
quantile_ouputs_diff

In [ ]:
plot_covid_scenarios_comparison(quantile_ouputs_diff, ['cumulative_diseased', 'cumulative_deaths'], [2021.0, 2022.0, 2025.0, 2035.0])